In [38]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [39]:
pair_nulos = pd.read_csv("files/pair_groupby.csv", index_col = 0)

In [40]:
pair_nulos.head()

,country,density(p/km2),abbreviation,agricultural_land(_%),land_area,armed_forces_size,birth_rate,calling_code,capital/major_city,co2-emissions,cpi,cpi_change_,currency-code,fertility_rate,forested_area_,gasoline_price,gdp,gross_primary_education_enrollment_,gross_tertiary_education_enrollment_,infant_mortality,largest_city,life_expectancy,maternal_mortality_ratio,minimum_wage,official_language,out_of_pocket_health_expenditure,physicians_per_thousand,population,population_labor_force_participation_,tax_revenue_,total_tax_rate,unemployment_rate,urban_population,latitud,longitud,continente
0,Afghanistan,60.0,AF,58.1,652230.0,323000.0,32.49,93.0,Kabul,8672.0,149.90,2.3,AFN,4.47,2.1,0.70,1.910135e+10,104.0,9.7,47.9,Kabul,64.5,638.0,0.43,Pashto,78.4,0.28,38041754.0,48.9,9.3,71.4,11.12,9797273.0,33.939110,67.709953,Asia
1,Albania,105.0,AL,43.1,28748.0,9000.0,11.78,355.0,Tirana,4536.0,119.05,1.4,ALL,1.62,28.1,1.36,1.527808e+10,107.0,55.0,7.8,Tirana,78.5,15.0,1.12,Albanian,56.9,1.20,2854191.0,55.7,18.6,36.6,12.33,1747593.0,41.153332,20.168331,Europe
2,Algeria,18.0,DZ,17.4,2381741.0,317000.0,24.28,213.0,Algiers,150006.0,151.36,2.0,DZD,3.02,0.8,0.28,1.699882e+11,109.9,51.4,20.1,Algiers,76.7,112.0,0.95,Arabic,28.1,1.72,43053054.0,41.2,37.2,66.1,11.70,31510100.0,28.033886,1.659626,Africa
3,Andorra,164.0,AD,40.0,468.0,NaN,7.20,376.0,Andorra la Vella,469.0,NaN,NaN,EUR,1.27,34.0,1.51,3.154058e+09,106.4,NaN,2.7,Andorra la Vella,NaN,NaN,6.63,Catalan,36.4,3.33,77142.0,NaN,NaN,NaN,NaN,67873.0,42.506285,1.521801,Europe
4,Angola,26.0,AO,47.5,1246700.0,117000.0,40.73,244.0,Luanda,34693.0,261.73,17.1,AOA,5.52,46.3,0.97,9.463542e+10,113.5,9.3,51.6,Luanda,60.8,241.0,0.71,Portuguese,33.4,0.21,31825295.0,77.5,9.2,49.1,6.89,21061025.0,-11.202692,17.873887,Africa


In [41]:
df_nulos = pd.DataFrame((pair_nulos.isnull().sum() / pair_nulos.shape[0]) * 100, columns = ["%_nulos"])
# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
abbreviation,3.589744
agricultural_land(_%),3.589744
land_area,0.512821
armed_forces_size,12.307692
birth_rate,3.076923
calling_code,0.512821
capital/major_city,1.538462
co2-emissions,3.589744
cpi,8.717949
cpi_change_,8.205128


In [42]:
nulos_esta_cat = pair_nulos[pair_nulos.columns[pair_nulos.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['abbreviation', 'capital/major_city', 'currency-code', 'largest_city',
       'official_language', 'continente'],
      dtype='object')


In [43]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(pair_nulos[col].value_counts() / pair_nulos.shape[0])
    print("........................")

La distribución de las categorías para la columna ABBREVIATION


AF    0.005128
PY    0.005128
NE    0.005128
NG    0.005128
KP    0.005128
        ...   
GR    0.005128
GD    0.005128
GT    0.005128
GN    0.005128
ZW    0.005128
Name: abbreviation, Length: 188, dtype: float64

........................
La distribución de las categorías para la columna CAPITAL/MAJOR_CITY


Kabul                    0.005128
Tirana                   0.005128
Wellington               0.005128
Managua                  0.005128
Niamey                   0.005128
                           ...   
Athens                   0.005128
St. George's, Grenada    0.005128
Guatemala City           0.005128
Conakry                  0.005128
Harare                   0.005128
Name: capital/major_city, Length: 192, dtype: float64

........................
La distribución de las categorías para la columna CURRENCY-CODE


...

........................
La distribución de las categorías para la columna OFFICIAL_LANGUAGE


English             0.158974
French              0.128205
Spanish             0.097436
Arabic              0.092308
Portuguese          0.035897
                      ...   
Albanian            0.005128
Jamaican English    0.005128
Lao                 0.005128
Latvian             0.005128
Shona               0.005128
Name: official_language, Length: 76, dtype: float64

........................
La distribución de las categorías para la columna CONTINENTE


Africa           0.271795
Asia             0.241026
Europe           0.230769
North America    0.117949
Oceania          0.071795
South America    0.061538
Name: continente, dtype: float64

........................


In [44]:
lista_unknown = ['abbreviation', 'capital/major_city', 'currency-code', 'largest_city',
       'official_language', 'continente']

In [45]:
for columna in lista_unknown:
    pair_nulos[columna] = pair_nulos[columna].fillna("Unknown")
    
pair_nulos[lista_unknown].isnull().sum()

abbreviation          0
capital/major_city    0
currency-code         0
largest_city          0
official_language     0
continente            0
dtype: int64

In [46]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_num = pair_nulos[pair_nulos.columns[pair_nulos.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['agricultural_land(_%)', 'land_area', 'armed_forces_size', 'birth_rate',
       'calling_code', 'co2-emissions', 'cpi', 'cpi_change_', 'fertility_rate',
       'forested_area_', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment_',
       'gross_tertiary_education_enrollment_', 'infant_mortality',
       'life_expectancy', 'maternal_mortality_ratio', 'minimum_wage',
       'out_of_pocket_health_expenditure', 'physicians_per_thousand',
       'population', 'population_labor_force_participation_', 'tax_revenue_',
       'total_tax_rate', 'unemployment_rate', 'urban_population', 'latitud',
       'longitud'],
      dtype='object')


In [47]:
# nulos que tenemos en cada una de las columnas numericas
pair_nulos[nulos_esta_num].isnull().sum() / pair_nulos.shape[0]

agricultural_land(_%)                    0.035897
land_area                                0.005128
armed_forces_size                        0.123077
birth_rate                               0.030769
calling_code                             0.005128
co2-emissions                            0.035897
cpi                                      0.087179
cpi_change_                              0.082051
fertility_rate                           0.035897
forested_area_                           0.035897
gasoline_price                           0.102564
gdp                                      0.010256
gross_primary_education_enrollment_      0.035897
gross_tertiary_education_enrollment_     0.061538
infant_mortality                         0.030769
life_expectancy                          0.041026
maternal_mortality_ratio                 0.071795
minimum_wage                             0.230769
out_of_pocket_health_expenditure         0.035897
physicians_per_thousand                  0.035897


In [64]:
for col in nulos_esta_num:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(pair_nulos[col].value_counts() / pair_nulos.shape[0])
    print("........................")

La distribución de las categorías para la columna AGRICULTURAL_LAND(_%)


17.4    0.015385
39.3    0.010256
23.3    0.010256
73.4    0.010256
34.5    0.010256
          ...   
58.0    0.005128
8.6     0.005128
66.8    0.005128
28.9    0.005128
41.9    0.005128
Name: agricultural_land(_%), Length: 168, dtype: float64

........................
La distribución de las categorías para la columna LAND_AREA


652230.0     0.005128
616.0        0.005128
268838.0     0.005128
130370.0     0.005128
1267000.0    0.005128
               ...   
349.0        0.005128
108889.0     0.005128
245857.0     0.005128
36125.0      0.005128
390757.0     0.005128
Name: land_area, Length: 194, dtype: float64

........................
La distribución de las categorías para la columna ARMED_FORCES_SIZE


...

........................
La distribución de las categorías para la columna LATITUD


 33.939110    0.005128
 17.357822    0.005128
-40.900557    0.005128
 12.865416    0.005128
 17.607789    0.005128
                ...   
 12.116500    0.005128
 15.783471    0.005128
 9.945587     0.005128
 11.803749    0.005128
-19.015438    0.005128
Name: latitud, Length: 194, dtype: float64

........................
La distribución de las categorías para la columna LONGITUD


 67.709953     0.005128
-62.782998     0.005128
 174.885971    0.005128
-85.207229     0.005128
 8.081666      0.005128
                 ...   
-61.679000     0.005128
-90.230759     0.005128
-9.696645      0.005128
-15.180413     0.005128
 29.154857     0.005128
Name: longitud, Length: 194, dtype: float64

........................


In [48]:
# Realizar copia DF
df_copia_ejercicios = pair_nulos.copy()

In [49]:
lista_num = ['agricultural_land(_%)', 'land_area', 'armed_forces_size', 'birth_rate',
       'calling_code', 'co2-emissions', 'cpi', 'cpi_change_', 'fertility_rate',
       'forested_area_', 'gasoline_price', 'gdp',
       'gross_primary_education_enrollment_',
       'gross_tertiary_education_enrollment_', 'infant_mortality',
       'life_expectancy', 'maternal_mortality_ratio', 'minimum_wage',
       'out_of_pocket_health_expenditure', 'physicians_per_thousand',
       'population', 'population_labor_force_participation_', 'tax_revenue_',
       'total_tax_rate', 'unemployment_rate', 'urban_population', 'latitud',
       'longitud']

In [50]:
# instanciamos las clases
clase_imputer = IterativeImputer(max_iter = 20, random_state = 42)
# ajustamos y tranformamos los datos
transformamos= clase_imputer.fit_transform(df_copia_ejercicios[lista_num])
# comprobamos que es lo que nos devuelve, que en este caso es un array también
transformamos

array([[ 5.8100000e+01,  6.5223000e+05,  3.2300000e+05, ...,
         9.7972730e+06,  3.3939110e+01,  6.7709953e+01],
       [ 4.3100000e+01,  2.8748000e+04,  9.0000000e+03, ...,
         1.7475930e+06,  4.1153332e+01,  2.0168331e+01],
       [ 1.7400000e+01,  2.3817410e+06,  3.1700000e+05, ...,
         3.1510100e+07,  2.8033886e+01,  1.6596260e+00],
       ...,
       [ 4.4600000e+01,  5.2796800e+05,  4.0000000e+04, ...,
         1.0869523e+07,  1.5552727e+01,  4.8516388e+01],
       [ 3.2100000e+01,  7.5261800e+05,  1.6000000e+04, ...,
         7.8717130e+06, -1.3133897e+01,  2.7849332e+01],
       [ 4.1900000e+01,  3.9075700e+05,  5.1000000e+04, ...,
         4.7173050e+06, -1.9015438e+01,  2.9154857e+01]])

In [51]:
lista_iterative = []
for col in lista_num:
    col = col + "_iterative"
    lista_iterative.append(col)
    

In [52]:
lista_iterative

['agricultural_land(_%)_iterative',
 'land_area_iterative',
 'armed_forces_size_iterative',
 'birth_rate_iterative',
 'calling_code_iterative',
 'co2-emissions_iterative',
 'cpi_iterative',
 'cpi_change__iterative',
 'fertility_rate_iterative',
 'forested_area__iterative',
 'gasoline_price_iterative',
 'gdp_iterative',
 'gross_primary_education_enrollment__iterative',
 'gross_tertiary_education_enrollment__iterative',
 'infant_mortality_iterative',
 'life_expectancy_iterative',
 'maternal_mortality_ratio_iterative',
 'minimum_wage_iterative',
 'out_of_pocket_health_expenditure_iterative',
 'physicians_per_thousand_iterative',
 'population_iterative',
 'population_labor_force_participation__iterative',
 'tax_revenue__iterative',
 'total_tax_rate_iterative',
 'unemployment_rate_iterative',
 'urban_population_iterative',
 'latitud_iterative',
 'longitud_iterative']

In [54]:
#en este caso como vamos a aplicar los dos métodos sobre estas columnas vamos a crear  columnas nuevas
df_copia_ejercicios[lista_iterative] = transformamos
print(df_copia_ejercicios[lista_iterative].isnull().sum())

agricultural_land(_%)_iterative                    0
land_area_iterative                                0
armed_forces_size_iterative                        0
birth_rate_iterative                               0
calling_code_iterative                             0
co2-emissions_iterative                            0
cpi_iterative                                      0
cpi_change__iterative                              0
fertility_rate_iterative                           0
forested_area__iterative                           0
gasoline_price_iterative                           0
gdp_iterative                                      0
gross_primary_education_enrollment__iterative      0
gross_tertiary_education_enrollment__iterative     0
infant_mortality_iterative                         0
life_expectancy_iterative                          0
maternal_mortality_ratio_iterative                 0
minimum_wage_iterative                             0
out_of_pocket_health_expenditure_iterative    

In [55]:
# KNNImputer.
# instanciamos la clase del KNNImputer
imputer_knn_ejercicios = KNNImputer(n_neighbors = 5)
# ajustamos y transformamos los datos
imputer_knn_imputado_ejer = imputer_knn_ejercicios.fit_transform(df_copia_ejercicios[lista_num])
# comprobamos que es lo que nos devuelve, que sigue siendo un array
imputer_knn_imputado_ejer

array([[ 5.8100000e+01,  6.5223000e+05,  3.2300000e+05, ...,
         9.7972730e+06,  3.3939110e+01,  6.7709953e+01],
       [ 4.3100000e+01,  2.8748000e+04,  9.0000000e+03, ...,
         1.7475930e+06,  4.1153332e+01,  2.0168331e+01],
       [ 1.7400000e+01,  2.3817410e+06,  3.1700000e+05, ...,
         3.1510100e+07,  2.8033886e+01,  1.6596260e+00],
       ...,
       [ 4.4600000e+01,  5.2796800e+05,  4.0000000e+04, ...,
         1.0869523e+07,  1.5552727e+01,  4.8516388e+01],
       [ 3.2100000e+01,  7.5261800e+05,  1.6000000e+04, ...,
         7.8717130e+06, -1.3133897e+01,  2.7849332e+01],
       [ 4.1900000e+01,  3.9075700e+05,  5.1000000e+04, ...,
         4.7173050e+06, -1.9015438e+01,  2.9154857e+01]])

In [56]:
lista_knn = []
for col in lista_num:
    col = col + "_knn"
    lista_knn.append(col)
    

In [57]:
lista_knn

['agricultural_land(_%)_knn',
 'land_area_knn',
 'armed_forces_size_knn',
 'birth_rate_knn',
 'calling_code_knn',
 'co2-emissions_knn',
 'cpi_knn',
 'cpi_change__knn',
 'fertility_rate_knn',
 'forested_area__knn',
 'gasoline_price_knn',
 'gdp_knn',
 'gross_primary_education_enrollment__knn',
 'gross_tertiary_education_enrollment__knn',
 'infant_mortality_knn',
 'life_expectancy_knn',
 'maternal_mortality_ratio_knn',
 'minimum_wage_knn',
 'out_of_pocket_health_expenditure_knn',
 'physicians_per_thousand_knn',
 'population_knn',
 'population_labor_force_participation__knn',
 'tax_revenue__knn',
 'total_tax_rate_knn',
 'unemployment_rate_knn',
 'urban_population_knn',
 'latitud_knn',
 'longitud_knn']

In [59]:
# Añadimos array al DF
df_copia_ejercicios[lista_knn] = imputer_knn_imputado_ejer

# comprobamos los nulos
print(df_copia_ejercicios[lista_knn].isnull().sum())

agricultural_land(_%)_knn                    0
land_area_knn                                0
armed_forces_size_knn                        0
birth_rate_knn                               0
calling_code_knn                             0
co2-emissions_knn                            0
cpi_knn                                      0
cpi_change__knn                              0
fertility_rate_knn                           0
forested_area__knn                           0
gasoline_price_knn                           0
gdp_knn                                      0
gross_primary_education_enrollment__knn      0
gross_tertiary_education_enrollment__knn     0
infant_mortality_knn                         0
life_expectancy_knn                          0
maternal_mortality_ratio_knn                 0
minimum_wage_knn                             0
out_of_pocket_health_expenditure_knn         0
physicians_per_thousand_knn                  0
population_knn                               0
population_la

In [61]:
lista_describe = []
for i in range(len(lista_num)):
    lista_describe.append(lista_num[i])
    lista_describe.append(lista_iterative[i])
    lista_describe.append(lista_knn[i])

In [62]:
df_copia_ejercicios.describe()[lista_describe]

,agricultural_land(_%),agricultural_land(_%)_iterative,agricultural_land(_%)_knn,land_area,land_area_iterative,land_area_knn,armed_forces_size,armed_forces_size_iterative,armed_forces_size_knn,birth_rate,birth_rate_iterative,birth_rate_knn,calling_code,calling_code_iterative,calling_code_knn,co2-emissions,co2-emissions_iterative,co2-emissions_knn,cpi,cpi_iterative,cpi_knn,cpi_change_,cpi_change__iterative,cpi_change__knn,fertility_rate,fertility_rate_iterative,fertility_rate_knn,forested_area_,forested_area__iterative,forested_area__knn,gasoline_price,gasoline_price_iterative,gasoline_price_knn,gdp,gdp_iterative,gdp_knn,gross_primary_education_enrollment_,gross_primary_education_enrollment__iterative,gross_primary_education_enrollment__knn,gross_tertiary_education_enrollment_,gross_tertiary_education_enrollment__iterative,gross_tertiary_education_enrollment__knn,infant_mortality,infant_mortality_iterative,infant_mortality_knn,life_expectancy,life_expectancy_iterative,life_expectancy_knn,maternal_mortality_ratio,maternal_mortality_ratio_iterative,maternal_mortality_ratio_knn,minimum_wage,minimum_wage_iterative,minimum_wage_knn,out_of_pocket_health_expenditure,out_of_pocket_health_expenditure_iterative,out_of_pocket_health_expenditure_knn,physicians_per_thousand,physicians_per_thousand_iterative,physicians_per_thousand_knn,population,population_iterative,population_knn,population_labor_force_participation_,population_labor_force_participation__iterative,population_labor_force_participation__knn,tax_revenue_,tax_revenue__iterative,tax_revenue__knn,total_tax_rate,total_tax_rate_iterative,total_tax_rate_knn,unemployment_rate,unemployment_rate_iterative,unemployment_rate_knn,urban_population,urban_population_iterative,urban_population_knn,latitud,latitud_iterative,latitud_knn,longitud,longitud_iterative,longitud_knn
count,188.000000,195.000000,195.000000,1.940000e+02,1.950000e+02,1.950000e+02,1.710000e+02,1.950000e+02,1.950000e+02,189.000000,195.000000,195.000000,194.000000,195.000000,195.000000,1.880000e+02,1.950000e+02,1.950000e+02,178.000000,195.000000,195.000000,179.000000,195.000000,195.000000,188.000000,195.000000,195.000000,188.000000,195.000000,195.000000,175.000000,195.000000,195.000000,1.930000e+02,1.950000e+02,1.950000e+02,188.000000,195.000000,195.000000,183.000000,195.000000,195.000000,189.000000,195.000000,195.000000,187.000000,195.000000,195.000000,181.000000,195.000000,195.000000,150.000000,195.000000,195.000000,188.000000,195.000000,195.000000,188.000000,195.000000,195.000000,1.940000e+02,1.950000e+02,1.950000e+02,176.000000,195.000000,195.000000,169.000000,195.000000,195.000000,183.000000,195.000000,195.000000,176.000000,195.000000,195.000000,1.900000e+02,1.950000e+02,1.950000e+02,194.000000,195.000000,195.000000,194.000000,195.000000,195.000000
mean,39.117553,39.100236,39.123077,6.896244e+05,6.896244e+05,6.888737e+05,1.592749e+05,1.458855e+05,1.407118e+05,20.214974,20.230006,20.275815,360.546392,360.546392,360.397949,1.777992e+05,1.709998e+05,1.715440e+05,190.460955,199.908405,186.947179,6.722346,7.243133,6.627590,2.698138,2.692279,2.713990,32.015426,32.032006,32.158872,1.002457,1.006490,1.016338,4.772959e+11,4.753266e+11,4.734768e+11,102.470213,102.466388,102.545744,37.963388,37.576611,37.026256,21.332804,21.208303,21.536923,72.279679,72.365632,72.102564,160.392265,155.308493,163.892308,2.194133,2.151966,2.145518,32.637234,32.785163,32.492308,1.839840,1.825123,1.828892,3.938116e+07,3.938116e+07,3.930265e+07,62.738068,62.765137,62.717231,16.573964,16.199473,16.631077,40.822404,40.832823,41.009641,6.886364,6.879384,6.983487,2.230454e+07,2.187044e+07,2.179509e+07,19.092351,19.018118,19.046831,20.232434,20.223160,20.191260
std,21.783052,21.387041,21.408945,1.921609e+06,1.916650e+06,1.916679e+06,3.806288e+05,3.581734e+05,3.598046e+05,9.945774,9.799636,9.868995,323.236419,322.402259,322.408923,8.387903e+05,8.243996e+05,8.241601e+05,397.947380,393.104331,380.363393,24.450414,24.105903,23.442692,1.282267,1.264884,1.26727

In [65]:
## Seleccionamos con que metodo nos quedamos y eliminamos columnas
#"co2-emissions_iterative", "cpi_iterative", "gross_tertiary_education_enrollment__iterative", 'maternal_mortality_ratio_iterative', "minimum_wage_iterative", 'tax_revenue__iterative' tiene min negativo
#para la mayoria nos sirven ambos metodos, por lo general iterative se acerca mas al original
#latitud y longitud dejamos los nulos ya que es un dato muy concreto que no puede conincidir

df_copia_ejercicios.drop(["agricultural_land(_%)", "agricultural_land(_%)_knn", "land_area", "land_area_knn", "armed_forces_size", "armed_forces_size_knn", "birth_rate", "birth_rate_knn", "calling_code", "calling_code_knn", "co2-emissions", "co2-emissions_iterative", "cpi", "cpi_iterative", "cpi_change_", "cpi_change__iterative", "fertility_rate", "fertility_rate_knn", "forested_area_",
 'forested_area__knn',
 'gasoline_price',
 'gasoline_price_knn',
 'gdp',
 'gdp_knn',
 'gross_primary_education_enrollment_',
 'gross_primary_education_enrollment__iterative',
 'gross_tertiary_education_enrollment_',
 'gross_tertiary_education_enrollment__iterative',
 'infant_mortality',
 'infant_mortality_iterative',
 'life_expectancy',
 'life_expectancy_knn',
 'maternal_mortality_ratio',
 'maternal_mortality_ratio_iterative',
 'minimum_wage',
 'minimum_wage_iterative',
 'out_of_pocket_health_expenditure',
 'out_of_pocket_health_expenditure_iterative',
 'physicians_per_thousand',
 'physicians_per_thousand_iterative',
 'population',
 'population_iterative',
 'population_labor_force_participation_',
 'population_labor_force_participation__iterative',
 'tax_revenue_',
 'tax_revenue__iterative',
 'total_tax_rate',
 'total_tax_rate_iterative',
 'unemployment_rate',
 'unemployment_rate_iterative',
 'urban_population',
 'urban_population_iterative',
 'latitud_iterative',
 'latitud_knn',
 'longitud_iterative',
 'longitud_knn'], axis = 1, inplace = True)



In [67]:
df_copia_ejercicios.columns

Index(['country', 'density(p/km2)', 'abbreviation', 'capital/major_city',
       'currency-code', 'largest_city', 'official_language', 'latitud',
       'longitud', 'continente', 'agricultural_land(_%)_iterative',
       'land_area_iterative', 'armed_forces_size_iterative',
       'birth_rate_iterative', 'calling_code_iterative',
       'fertility_rate_iterative', 'forested_area__iterative',
       'gasoline_price_iterative', 'gdp_iterative',
       'life_expectancy_iterative', 'co2-emissions_knn', 'cpi_knn',
       'cpi_change__knn', 'gross_primary_education_enrollment__knn',
       'gross_tertiary_education_enrollment__knn', 'infant_mortality_knn',
       'maternal_mortality_ratio_knn', 'minimum_wage_knn',
       'out_of_pocket_health_expenditure_knn', 'physicians_per_thousand_knn',
       'population_knn', 'population_labor_force_participation__knn',
       'tax_revenue__knn', 'total_tax_rate_knn', 'unemployment_rate_knn',
       'urban_population_knn'],
      dtype='object')

In [68]:
dicc = {'agricultural_land(_%)_iterative' : 'agricultural_land',
       'land_area_iterative' : 'land_area', 'armed_forces_size_iterative' : 'armed_forces_size',
       'birth_rate_iterative' : 'birth_rate', 'calling_code_iterative' : 'calling_code',
       'fertility_rate_iterative' : 'fertility_rate', 'forested_area__iterative' : 'forested_area',
       'gasoline_price_iterative' : 'gasoline_price', 'gdp_iterative' : 'gdp',
       'life_expectancy_iterative' : 'life_expectancy', 'co2-emissions_knn' : 'co2-emissions', 'cpi_knn' : 'cpi',
       'cpi_change__knn' : 'cpi_change', 'gross_primary_education_enrollment__knn' : 'gross_primary_education_enrollment',
       'gross_tertiary_education_enrollment__knn' : 'gross_tertiary_education_enrollment', 'infant_mortality_knn' : 'infant_mortality',
       'maternal_mortality_ratio_knn' : 'maternal_mortality_ratio', 'minimum_wage_knn' : 'minimum_wage',
       'out_of_pocket_health_expenditure_knn' : 'out_of_pocket_health_expenditure', 'physicians_per_thousand_knn' : 'physicians_per_thousand',
       'population_knn' : 'population', 'population_labor_force_participation__knn' : 'population_labor_force_participation',
       'tax_revenue__knn' : 'tax_revenue', 'total_tax_rate_knn' : 'total_tax_rate', 'unemployment_rate_knn' : 'unemployment_rate',
       'urban_population_knn' : 'urban_population'}

In [69]:
# Key valor actual, Value nuevo nombre (nombre que mantenian antes para que cuadre con la documentacion)
nuevo_nombre = dicc

df_copia_ejercicios.rename(columns = nuevo_nombre, inplace = True)


In [72]:
df_copia_ejercicios[df_copia_ejercicios.columns].isnull().sum()

country                                 0
density(p/km2)                          0
abbreviation                            0
capital/major_city                      0
currency-code                           0
largest_city                            0
official_language                       0
latitud                                 1
longitud                                1
continente                              0
agricultural_land                       0
land_area                               0
armed_forces_size                       0
birth_rate                              0
calling_code                            0
fertility_rate                          0
forested_area                           0
gasoline_price                          0
gdp                                     0
life_expectancy                         0
co2-emissions                           0
cpi                                     0
cpi_change                              0
gross_primary_education_enrollment

In [73]:
# Guardamos el DataFrame para usarlo en la lección de mañana 
df_copia_ejercicios.to_csv("files/df-clean-nonulls.csv")